### 재무제표 주석에서 특정 Section 찾기

* 문서 URL 목록 가져오기

In [225]:
import pandas as pd

# 사업보고서 제무제표 주석 문서 URL 목록
file_path_url = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path_url, dtype=object)
# 제조업 사업보고서 목록
file_path_corp = "D:/PythonProject/data-gatherer/dart_fs_notes/company/"
df_info = pd.read_csv(file_path_corp + 'corp_info_all.csv', dtype=object)
# "제조업"만 리스트에 담는다
df_all_info = pd.merge(fs_notes_all, df_info[['corp_code', 'induty_code']], how='left', left_on='corp_code', right_on='corp_code')
df_mfg_fs_notes = df_all_info[df_all_info['induty_code'].str[0:2].between('10', '34', inclusive='both')]

# 회사코드 정렬
df_mfg_fs_notes = df_mfg_fs_notes[df_mfg_fs_notes.rpt_num.str.startswith('2017')]
df_mfg_fs_notes = df_mfg_fs_notes.sort_values(by=['corp_code'], ascending=True)
# 보고서 목록
list_all_codes = df_mfg_fs_notes.rpt_num.unique()
len(list_all_codes)


241

* 리스트 분할

In [226]:
list_target_codes = list_all_codes
list_cnt = len(list_target_codes)
n = 100
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

3

* 재무제표 주석 문서에서 '퇴직연금' 관련 Section 영역 찾기

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# 검색 우선순위를 고려한 순서로 리스트 구성
TARGET_KEYWORDS = [
    '순확정급여부채', '순확정급여채무', '순확정급여자산',
    '퇴직급여부채', '퇴직급여채무', '퇴직급여제도',    
    '퇴직급여충당부채',
    '퇴직급여',
    '확정급여부채', '확정급여채무', '확정급여제도',
    '종업원급여', '종업원 급여',
    '비용의 성격별 내역', '비용의 성격별 분류', '수익과 비용의 분석',
    '기타부채',
]
        
def cleanPage(page):
    # span tag 제거
    # cleanr = re.compile('<(span|SPAN)[^>]*')  # 정규표현식  
    # page = re.sub(cleanr, '', page)
    page = re.sub('<(span|SPAN).*?>', '', page, 0).strip()
    page = re.sub('</(span|SPAN)>', '', page, 0).strip()
        
    # 줄바꿈없는 공백(Non-breaking Space) 표식 제거
    page = page.replace("&nbsp;", " ")
    # page = page.replace(u'\xa0', '')
    # page = page.replace('\xa0', ' ')
    # page = page.replace("\xc2\xa0", " ")    
    
    # re.sub("\d+[.]+\s+[^0-9]+", "\d+[.]+[^0-9]+", page)
    # re.sub("\s+$", "", page)
    """
    줄바꿈(Line Breaks) 제거
    \r\n - on a windows computer
    \r - on an Apple computer
    \n - on Linux
    """
    # page = page.replace("\r\n", "")
    # page = page.replace("\r", "")
    # page = page.replace("\n", "")
    # strip out all tailing whitespace
    # for line in page:
    #     line = line.rstrip()    
    # page = re.sub('\n\n', '', page)
    # page = ''.join(page.splitlines())    
    
    # str_list = [p.rstrip('\n') for p in page]
    # page = "".join(str_list)
    
    return page

def get_document(doc_url):
    # 재무제표 주석 url의 문서 가져오기
    response = requests.get(doc_url)
    if response.status_code != 200:     # URL GET '200 정상'
        return None
        
    page = cleanPage(response.text)
    # soup = BeautifulSoup(page, features="lxml")
    soup = BeautifulSoup(page, 'html.parser')
    return soup
    
def get_subtitles(soup):        
    # tags = soup.find_all('p')
    tags = soup.find_all(string = True)
    subtitle_list = []
    prev_number = 0
    for t in tags:
        subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
        subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)  # 숫자+'.'+숫자가아닌모든문자열
        subjects_all = subjects1 + subjects2
        subjects = list(set(subjects_all))      # 중복 제거
        if len(subjects) > 0:
            for s in subjects:
                subtitle_list.append(s)               
    
    ordered_subtitle_list = []
    prev_no = 0             
    for s in subtitle_list:
        this_no = int(s.split(".")[0])
        if this_no == prev_no + 1 or this_no == prev_no + 2 or this_no == prev_no + 3:
            ordered_subtitle_list.append(s)
            prev_no = this_no

    return ordered_subtitle_list

def find_section_title(subtitle_list):
    if len(subtitle_list) == 0:
        return None
    
    section_title = None
    for keyword in TARGET_KEYWORDS:
        if section_title is None:
            for subtitle in subtitle_list:
                if subtitle.__contains__(keyword):
                    section_title = subtitle
                    break
        else:
            break    

    if section_title is None:
        return None
    
    return section_title

def get_section_html(soup, subtitle_list, section_title):
    idx = 0
    idx = subtitle_list.index(section_title)
    next_title = subtitle_list[idx + 1]
    # 문서에서 해당 Section 시작위치와 종료위치 찾기
    doc_html = soup.prettify()
    p0 = doc_html.find(section_title)
    p1 = doc_html.find(next_title)
    section_html = doc_html[p0:p1]     # 문서 Slicing
    return section_html

In [228]:
import traceback
from time import sleep
from tqdm import tqdm

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/section/"

for reports in list_of_lists:
    data_list = []
    for rpt_num in tqdm(reports):
        sleep(1.0)
        df_doc = fs_notes_all.query("rpt_num == @rpt_num")
        for i, doc in df_doc.iterrows():
            doc_url = doc['doc_url']
            if isinstance(doc_url, str) and doc_url != None:
                try:
                    soup = get_document(doc_url)
                    subtitle_list = get_subtitles(soup)
                    section_title = find_section_title(subtitle_list)
                    if section_title is None:      
                        data_list.append({'corp_code' : doc['corp_code'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 'doc_title' : doc['doc_title'], 'doc_result' : doc['doc_result'], 
                                        'section_title' : None, 'doc_url' : doc_url})
                    else:
                        data_list.append({'corp_code' : doc['corp_code'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 'doc_title' : doc['doc_title'], 'doc_result' : doc['doc_result'], 
                                        'section_title' : section_title, 'doc_url' : doc_url})
                except ValueError as err:
                    print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc_url)
                    print(err)
                    break
                except Exception as e:
                    print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc_url)
                    err_msg = traceback.format_exc()
                    print(err_msg)
                    break
            else:
                data_list.append({'corp_code' : doc['corp_code'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 'doc_title' : doc['doc_title'], 'doc_result' : doc['doc_result'], 
                                    'section_title' : None, 'doc_url' : doc_url})
    
    # 결과를 파일로 저장
    if len(data_list) > 0:
        df_sections = pd.DataFrame(data_list)
        file_name = 'section_' + myutil.now_dt_str() + '.csv'
        df_sections.to_csv(path_dir + file_name, index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 41/41 [01:10<00:00,  1.71s/it]


In [229]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/section/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'sections_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'sections_all.csv', index=False)

In [230]:
df_all.shape[0]

1641

In [207]:
import pandas as pd
df1 = df_mfg_fs_notes
df2 = df_all
# df1에는 있으나 df2에는 데이터가 빠져있는 목록
df_missing = pd.merge(df1, df2, how='outer', left_on='rpt_num', right_on='rpt_num' , indicator=True).query("_merge=='left_only'")

In [208]:
df_missing

,corp_code_x,stock_code,corp_cls,corp_name_x,rpt_num,rpt_name_x,rpt_url,doc_title_x,doc_url_x,doc_result_x,induty_code,corp_code_y,corp_name_y,rpt_name_y,doc_title_y,doc_result_y,section_title,doc_url_y,_merge


In [231]:
# pd.options.display.max_colwidth = 50       # default
pd.options.display.max_colwidth = 300
# Section Title을 찾지 못한 건들
df_err = df_all.loc[(df_all.doc_result == 'OK') & (df_all.section_title.isna()), ]
df_err.shape[0]

1

In [232]:
df_err

,corp_code,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,doc_url
909,00113544,대한화섬,20190417000551,[기재정정]사업보고서 (2014.12),5. 재무제표 주석,OK,NaN,http://dart.fss.or.kr/report/viewer.do?rcpNo=20190417000551&dcmNo=6703167&eleId=18&offset=240127&length=266691&dtd=dart3.xsd


In [211]:
df_doc_urls = df_err['doc_url']

In [212]:
title_list = []
for doc_url in df_doc_urls:
    # doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20220324000331&dcmNo=8501290&eleId=22&offset=668365&length=244912&dtd=dart3.xsd'
    response = requests.get(doc_url)
    page = cleanPage(response.text)
    # soup = BeautifulSoup(page, features="lxml")
    soup = BeautifulSoup(page, 'html.parser')
    
    tags = soup.find_all(string = True)
    subtitle_list = []
    prev_number = 0
    for t in tags:
        # print(t.text)
        # subjects = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
        subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
        subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)  # 숫자+'.'+숫자가아닌모든문자열
        subjects_all = subjects1 + subjects2
        subjects = list(set(subjects_all))      # 중복 제거
        if len(subjects) > 0:
            for s in subjects:
                subtitle_list.append(s)
        
    ordered_subtitle_list = []
    prev_no = 0             
    for s in subtitle_list:
        this_no = int(s.split(".")[0])
        if this_no == prev_no + 1 or this_no == prev_no + 2 or this_no == prev_no + 3:
            ordered_subtitle_list.append(s)
            prev_no = this_no
    
    title_list.append({'doc_url' : doc_url, 'title_list' : ordered_subtitle_list})
                      
df_title_list = pd.DataFrame(title_list)                      


In [213]:
pd.options.display.max_colwidth = 1000
df_title_list

,doc_url,title_list
0,http://dart.fss.or.kr/report/viewer.do?rcpNo=20190417000551&dcmNo=6703167&eleId=18&offset=240127&length=266691&dtd=dart3.xsd,[]


* 테스트

In [7]:
doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402003544&dcmNo=6053977&eleId=16&offset=651509&length=295042&dtd=dart3.xsd'
response = requests.get(doc_url)
page = cleanPage(response.text)
# soup = BeautifulSoup(page, features="lxml")
soup = BeautifulSoup(page, 'html.parser')

tags = soup.find_all(string = True)
subtitle_list = []
prev_number = 0
for t in tags:
    # print(t.text)
    # subjects = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
    subjects1 = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열     
    subjects2 = re.findall('\d+[.]+[^0-9]+', t.text)  # 숫자+'.'+숫자가아닌모든문자열
    subjects_all = subjects1 + subjects2
    subjects = list(set(subjects_all))      # 중복 제거
    if len(subjects) > 0:
        for s in subjects:
            subtitle_list.append(s)
    
ordered_subtitle_list = []
prev_no = 0             
for s in subtitle_list:
    this_no = int(s.split(".")[0])
    if this_no == prev_no + 1 or this_no == prev_no + 2 or this_no == prev_no + 3:
        ordered_subtitle_list.append(s)
        prev_no = this_no                   

In [6]:
ordered_subtitle_list

['1. 회사의 개요',
 '2. 재무제표 작성기준 및 중요한 회계정책',
 '3. 중요한 회계적 판단, 추정 및 가정',
 '4. 제정ㆍ공표되었으나 아직 시행되지 않은 회계기준',
 '5. 영업부문',
 '6. 현금및현금성자산',
 '7. 매출채권및기타채권',
 '8. 재고자산 ',
 '9. 금융자산',
 '10. 금융부채',
 '11. 기타유동자산',
 '12. 종속기업 및 관계기업투자',
 '13. 유형자산',
 '14. 투자부동산',
 '15. 무형자산',
 '16. 매입채무및기타채무',
 '17. 차입금',
 '18. 기타유동부채',
 '19. 파생상품',
 '20. 퇴직급여  ',
 '21. 외화자산ㆍ부채',
 '22. 자본금과 잉여금',
 '23. 영업이익',
 '24. 금융손익',
 '25. 기타영업외손익',
 '26. 특수관계자 공시',
 '27. 법인세',
 '28. 주당이익',
 '29. 현금흐름표',
 '30. 우발채무와 약정사항',
 '31. 재무위험관리의 목적 및 정책',
 '32. 공정가치',
 '33. 매각예정자산',
 '34. 코로나',
 '35. 보고기간후사건 - 우크라이나 사태 영향의 불확실성',
 '36. 재무제표의 사실상 확정된 날과 승인기관 ']

In [158]:
section_title = None
for keyword in TARGET_KEYWORDS:
    print(keyword)
    if section_title is None:
        for subtitle in subtitle_list:
            if subtitle.__contains__(keyword):
                print(subtitle)
                section_title = subtitle
                break    

순확정급여부채
순확정급여채무
순확정급여자산
퇴직급여부채
퇴직급여채무
퇴직급여제도
17. 퇴직급여제도
퇴직급여충당부채
퇴직급여
확정급여부채
확정급여채무
확정급여제도
종업원급여
종업원 급여
비용의 성격별 내역
비용의 성격별 분류
기타부채


In [159]:
section_title

'17. 퇴직급여제도'

In [153]:
v = find_section_title2(ordered_subtitle_list)
v

'14. 기타채무 및 기타부채'

In [121]:
doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321000425&dcmNo=8481261&eleId=22&offset=648094&length=351537&dtd=dart3.xsd'
response = requests.get(doc_url)
page = re.sub('<.+?>', '', response.text, 0).strip()
soup = BeautifulSoup(page, features="lxml")

In [63]:
import requests
from bs4 import BeautifulSoup
import re
doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321000425&dcmNo=8481261&eleId=22&offset=648094&length=351537&dtd=dart3.xsd'
response = requests.get(doc_url)
page = cleanPage(response.text)
# soup = BeautifulSoup(page, features="lxml")
# page = re.sub('<span.+?>', '', response.text, 0).strip()
# page = re.sub('<(span|SPAN).*?>', '', response.text, 0).strip()
soup = BeautifulSoup(page, features="lxml")

In [64]:
import pandas as pd
df = pd.read_html(page)

In [66]:
page

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<HTML style=\'border:0\'>\r\n<HEAD>\r\n<TITLE></TITLE>\r\n<meta http-equiv="X-UA-Compatible" content="ie=edge" >\r\n<meta http-equiv="X-UA-TextLayoutMetrics" content="gdi" >\r\n<META http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<link rel="stylesheet" type="text/css" href="/css/report_xml.css">\r\n</HEAD>\r\n<BODY bgcolor="#FFFFFF">\r\n<P><BR/></P>\r\n<P class=\'section-2\'><A name=\'toc1\'>5. 재무제표 주석</A></P>\r\n<P style=\'font-family:DartNBSP, "굴림", Gulim;font-size:16pt;color:#0000FF;font-weight:bold;\'><BR/></P>\r\n<TABLE class=\'nb\' width=\'600\'>\r\n<COLGROUP>\r\n<COL width=\'600\'></COL>\r\n</COLGROUP>\r\n<TBODY>\r\n<TR>\r\n  <TD width=\'600\' height=\'20\' align=\'CENTER\' valign=\'TOP\'>제5(당)기말 2021년 12월 31일 현재</TD>\r\n</TR>\r\n<TR>\r\n  <TD width=\'600\' height=\'20\' align=\'CENTER\' valign=\'TOP\'>제4(전)기말 2020년 12월 31일 현재</TD>\r\n</TR>\r\n<TR>\r\n  <T

In [60]:
df

[                          0
 0  제5(당)기말 2021년 12월 31일 현재
 1  제4(전)기말 2020년 12월 31일 현재
 2               동아타이어공업주식회사,
            주주명        당기                전기        
            주주명    주식수(주)  지분율(%)    주식수(주)  지분율(%)
 0          김만수   3928419   28.61   3928419   28.61
 1          김상헌   1750297   12.75   1750297   12.74
 2    ㈜디티알오토모티브   1738609   12.66   1738609   12.66
 3          김민찬   1380000   10.05   1380000   10.05
 4  한국투자밸류자산운용㈜    660281    4.81   1857844   13.53
 5           기타   4273811   31.12   3076248   22.40
 6         자기주식      1598    0.01      1598    0.01
 7           합계  13733015  100.00  13733015  100.00,
        구 분                               구 분.1  \
 0  Stage 1  최초 인식 후 신용위험이 유의적으로 증가하지 않은 경우(주1)   
 1  Stage 2          최초 인식 후 신용위험이 유의적으로 증가한 경우   
 2  Stage 3                          신용이 손상된 경우   
 
                                                               손실충당금  
 0  12개월 기대신용손실: 보고기간 말 이후 12개월 내에 발생 가능한 금융상품의 채무불이행 사건으로 인한 기대신용손실  
 1            

In [58]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html style="border:0">
<head>
<title></title>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">5. 재무제표 주석</a></p>
<p style='font-family:DartNBSP, "굴림", Gulim;font-size:16pt;color:#0000FF;font-weight:bold;'><br/></p>
<table class="nb" width="600">
<colgroup>
<col width="600"/>
</colgroup>
<tbody>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제5(당)기말 2021년 12월 31일 현재</td>
</tr>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600">제4(전)기말 2020년 12월 31일 현재</td>
</tr>
<tr>
<td align="CENTER" height="20" valign="TOP" width="600"><br/></td>
</tr>
<tr>
<td height="20" valign="TOP" width="600

In [84]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')  # 정규표현식
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [33]:
import re

def cleanhtml(raw_html):
  # cleanr = re.compile('<span.+?>')  # 정규표현식
  cleanr = re.compile('<(span|SPAN)[^>]*')  # 정규표현식  
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [34]:
import requests
from bs4 import BeautifulSoup
import re
doc_url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20220321000425&dcmNo=8481261&eleId=22&offset=648094&length=351537&dtd=dart3.xsd'
response = requests.get(doc_url)
page = cleanhtml(response.text)
soup = BeautifulSoup(page, features="lxml")